In [1]:
#!/usr/bin/env python3
"""
Week 4: Cell Type Annotation for GSE116222
==========================================

Two approaches:
1. Marker-based: Use known cell type markers to identify clusters
2. Differential expression: Find cluster-defining genes

Key concept:
Cell type annotation is part science, part art. You're matching gene expression
patterns to known biology. Multiple lines of evidence strengthen annotations.
"""

import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

# Settings
sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=False)

# Paths
PROCESSED_DIR = Path("../data/processed")
FIGURES_DIR = Path("../figures")
RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ==============================================================================
# Step 1: Load Clustered Data
# ==============================================================================

print("="*60)
print("LOADING CLUSTERED DATA")
print("="*60)

adata = sc.read_h5ad(PROCESSED_DIR / "GSE116222_clustered.h5ad")
print(f"Loaded: {adata.n_obs} cells × {adata.n_vars} genes")
print(f"Clusters: {adata.obs['leiden'].nunique()}")

# ==============================================================================
# Step 2: Define Marker Genes
# ==============================================================================

print("\n" + "="*60)
print("DEFINING MARKER GENE PANELS")
print("="*60)

# Literature-based markers for colonic epithelial cell types
# Sources: Parikh et al. 2019 Nature, Smillie et al. 2019 Cell

MARKERS = {
    # Stem and progenitor cells
    'Stem cells': ['LGR5', 'ASCL2', 'SMOC2', 'RGMB', 'OLFM4'],
    'TA (Transit-amplifying)': ['MKI67', 'TOP2A', 'PCNA', 'MCM5'],
    
    # Absorptive lineage
    'Enterocytes': ['CA1', 'CA2', 'SLC26A3', 'FABP1', 'FABP2'],
    'BEST4+ enterocytes': ['BEST4', 'OTOP2', 'CA7', 'SPIB'],
    'Colonocytes': ['AQP8', 'SLC26A2', 'CEACAM7'],
    
    # Secretory lineage
    'Goblet cells': ['MUC2', 'TFF3', 'FCGBP', 'ZG16', 'CLCA1'],
    'Goblet (WFDC2+)': ['WFDC2', 'TFF1', 'AGR2'],
    'Enteroendocrine': ['CHGA', 'CHGB', 'PYY', 'GCG', 'TPH1', 'NEUROD1'],
    'Tuft cells': ['DCLK1', 'TRPM5', 'GFI1B', 'POU2F3'],
    
    # Inflammation/immune-related
    'Immune infiltrate': ['PTPRC', 'CD3D', 'CD3E', 'CD8A', 'CD4'],
    'Inflammatory response': ['CXCL1', 'CXCL8', 'IL1B', 'TNFAIP3', 'NFKBIA'],
    'Interferon response': ['ISG15', 'MX1', 'IFIT1', 'IFI6', 'OAS1'],
    
    # Other markers
    'Paneth-like': ['LYZ', 'DEFA5', 'DEFA6', 'REG3A'],
    'M cells': ['GP2', 'SPIB', 'CCL20'],
}

# Check which markers are present in our dataset
print("\nMarker availability:")
for cell_type, genes in MARKERS.items():
    present = [g for g in genes if g in adata.var_names]
    print(f"  {cell_type}: {len(present)}/{len(genes)} markers")

# ==============================================================================
# Step 3: Visualize Marker Expression on UMAP
# ==============================================================================

print("\n" + "="*60)
print("VISUALIZING KEY MARKERS ON UMAP")
print("="*60)

# Select key markers for visualization (one per major cell type)
key_markers = [
    'LGR5',    # Stem
    'MKI67',   # Proliferating
    'CA1',     # Enterocyte
    'BEST4',   # BEST4+ cells
    'MUC2',    # Goblet
    'CHGA',    # Enteroendocrine
    'WFDC2',   # WFDC2+ goblet
    'PTPRC',   # Immune (CD45)
]

# Filter to markers present in data
key_markers = [m for m in key_markers if m in adata.var_names]
print(f"Plotting markers: {key_markers}")

# Create UMAP plots for key markers
n_markers = len(key_markers)
n_cols = 4
n_rows = (n_markers + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 4*n_rows))
axes = axes.flatten()

for i, marker in enumerate(key_markers):
    sc.pl.umap(adata, color=marker, ax=axes[i], show=False, 
               title=marker, vmax='p99', cmap='Reds')

# Hide empty subplots
for i in range(len(key_markers), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(FIGURES_DIR / "12_marker_umap.png", dpi=150, bbox_inches='tight')
print(f"Saved: {FIGURES_DIR}/12_marker_umap.png")
plt.show()

# ==============================================================================
# Step 4: Dotplot of Markers by Cluster
# ==============================================================================

print("\n" + "="*60)
print("CREATING MARKER DOTPLOT")
print("="*60)

# Flatten marker list and filter to present genes
all_markers = []
for genes in MARKERS.values():
    all_markers.extend(genes)
all_markers = [g for g in all_markers if g in adata.var_names]
all_markers = list(dict.fromkeys(all_markers))  # Remove duplicates, preserve order

print(f"Total markers for dotplot: {len(all_markers)}")

# Create dotplot
fig, ax = plt.subplots(figsize=(20, 8))
sc.pl.dotplot(
    adata, 
    var_names=all_markers,
    groupby='leiden',
    dendrogram=True,
    ax=ax,
    show=False,
    standard_scale='var'  # Scale each gene 0-1
)
plt.tight_layout()
plt.savefig(FIGURES_DIR / "13_marker_dotplot.png", dpi=150, bbox_inches='tight')
print(f"Saved: {FIGURES_DIR}/13_marker_dotplot.png")
plt.show()

# ==============================================================================
# Step 5: Find Cluster Marker Genes (Differential Expression)
# ==============================================================================

print("\n" + "="*60)
print("FINDING CLUSTER MARKER GENES")
print("="*60)

# Wilcoxon rank-sum test: each cluster vs all other cells
# Uses raw counts stored in layers for accurate statistics

sc.tl.rank_genes_groups(
    adata,
    groupby='leiden',
    method='wilcoxon',
    pts=True,  # Calculate percentage of cells expressing
    key_added='rank_genes_leiden'
)

print("Computed differential expression for all clusters")

# Visualize top markers per cluster
fig, ax = plt.subplots(figsize=(12, 10))
sc.pl.rank_genes_groups(adata, n_genes=10, key='rank_genes_leiden', 
                         ax=ax, show=False)
plt.tight_layout()
plt.savefig(FIGURES_DIR / "14_cluster_markers_ranked.png", dpi=150, bbox_inches='tight')
print(f"Saved: {FIGURES_DIR}/14_cluster_markers_ranked.png")
plt.show()

# ==============================================================================
# Step 6: Extract and Save Marker Gene Tables
# ==============================================================================

print("\n" + "="*60)
print("EXTRACTING MARKER GENE TABLES")
print("="*60)

# Get top markers for each cluster
def get_marker_table(adata, key='rank_genes_leiden', n_genes=50):
    """Extract marker genes as a DataFrame."""
    result = adata.uns[key]
    groups = result['names'].dtype.names
    
    tables = []
    for group in groups:
        df = pd.DataFrame({
            'gene': result['names'][group][:n_genes],
            'score': result['scores'][group][:n_genes],
            'logfoldchange': result['logfoldchanges'][group][:n_genes],
            'pval': result['pvals'][group][:n_genes],
            'pval_adj': result['pvals_adj'][group][:n_genes],
            'pct_in': result['pts'][group][:n_genes],
            'pct_out': result['pts_rest'][group][:n_genes],
        })
        df['cluster'] = group
        tables.append(df)
    
    return pd.concat(tables, ignore_index=True)

marker_df = get_marker_table(adata)
marker_df.to_csv(RESULTS_DIR / "cluster_markers.csv", index=False)
print(f"Saved: {RESULTS_DIR}/cluster_markers.csv")

# Print top 5 markers per cluster
print("\nTop 5 marker genes per cluster:")
print("-" * 60)
for cluster in sorted(adata.obs['leiden'].unique(), key=int):
    subset = marker_df[marker_df['cluster'] == cluster].head(5)
    genes = ', '.join(subset['gene'].values)
    print(f"Cluster {cluster}: {genes}")

# ==============================================================================
# Step 7: Heatmap of Top Markers
# ==============================================================================

print("\n" + "="*60)
print("CREATING MARKER HEATMAP")
print("="*60)

# Get top 3 markers per cluster for heatmap
top_markers_per_cluster = {}
for cluster in adata.obs['leiden'].unique():
    subset = marker_df[marker_df['cluster'] == cluster].head(3)
    top_markers_per_cluster[cluster] = subset['gene'].tolist()

# Flatten and deduplicate
heatmap_genes = []
for cluster in sorted(top_markers_per_cluster.keys(), key=int):
    for gene in top_markers_per_cluster[cluster]:
        if gene not in heatmap_genes:
            heatmap_genes.append(gene)

print(f"Genes for heatmap: {len(heatmap_genes)}")

# Create heatmap
sc.pl.heatmap(
    adata,
    var_names=heatmap_genes,
    groupby='leiden',
    swap_axes=True,
    show_gene_labels=True,
    standard_scale='var',
    figsize=(12, 14),
    show=False
)
plt.savefig(FIGURES_DIR / "15_marker_heatmap.png", dpi=150, bbox_inches='tight')
print(f"Saved: {FIGURES_DIR}/15_marker_heatmap.png")
plt.show()

# ==============================================================================
# Step 8: Score Cell Types
# ==============================================================================

print("\n" + "="*60)
print("SCORING CELL TYPE SIGNATURES")
print("="*60)

# Score each cell for each cell type signature
for cell_type, genes in MARKERS.items():
    # Filter to genes present in data
    genes_present = [g for g in genes if g in adata.var_names]
    if len(genes_present) >= 2:  # Need at least 2 genes for scoring
        score_name = cell_type.replace(' ', '_').replace('(', '').replace(')', '').replace('+', 'pos')
        sc.tl.score_genes(adata, gene_list=genes_present, score_name=score_name)
        print(f"  Scored: {cell_type} ({len(genes_present)} genes)")

# Visualize scores on UMAP
score_columns = [c for c in adata.obs.columns if c.endswith('1') or 
                 any(x in c for x in ['Stem', 'Goblet', 'Entero', 'BEST4', 'Immune', 'Inflam'])]
score_columns = [c for c in adata.obs.columns if c in 
                 ['Stem_cells1', 'TA_Transit-amplifying1', 'Enterocytes1', 
                  'BEST4pos_enterocytes1', 'Goblet_cells1', 'Enteroendocrine1',
                  'Immune_infiltrate1', 'Inflammatory_response1']]

# Find actual score column names
score_cols = [c for c in adata.obs.columns if any(ct.replace(' ', '_').replace('(', '').replace(')', '').replace('+', 'pos') in c 
              for ct in MARKERS.keys())]

if score_cols:
    n_scores = min(8, len(score_cols))
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for i, score in enumerate(score_cols[:n_scores]):
        sc.pl.umap(adata, color=score, ax=axes[i], show=False,
                   title=score.replace('1', ''), cmap='RdBu_r', vcenter=0)
    
    for i in range(n_scores, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / "16_celltype_scores.png", dpi=150, bbox_inches='tight')
    print(f"Saved: {FIGURES_DIR}/16_celltype_scores.png")
    plt.show()

# ==============================================================================
# Step 9: Preliminary Annotation
# ==============================================================================

print("\n" + "="*60)
print("PRELIMINARY CLUSTER ANNOTATIONS")
print("="*60)

# Based on marker expression and cluster composition, create initial annotations
# This is a starting point - you'll refine based on the visualizations

print("""
ANNOTATION GUIDELINES:

Look at your dotplot and marker UMAPs to fill in this table.
For each cluster, identify:
1. Which marker genes are highly expressed
2. The condition composition (from Week 3)
3. Assign a cell type label

Template for annotation (you'll fill this in):
""")

# Print cluster summary to help with annotation
cluster_summary = adata.obs.groupby('leiden').agg({
    'condition': lambda x: x.value_counts().to_dict(),
    'n_genes_by_counts': 'median',
    'total_counts': 'median'
})

print("\nCluster statistics for annotation reference:")
print("-" * 80)

for cluster in sorted(adata.obs['leiden'].unique(), key=int):
    condition_dist = adata.obs[adata.obs['leiden'] == cluster]['condition'].value_counts(normalize=True) * 100
    n_cells = (adata.obs['leiden'] == cluster).sum()
    
    # Get top markers
    top_genes = marker_df[marker_df['cluster'] == cluster].head(5)['gene'].tolist()
    
    print(f"\nCluster {cluster} (n={n_cells}):")
    print(f"  Top markers: {', '.join(top_genes)}")
    print(f"  Composition: ", end='')
    for cond in ['Healthy', 'UC_noninflamed', 'UC_inflamed']:
        pct = condition_dist.get(cond, 0)
        print(f"{cond[:7]}:{pct:.0f}% ", end='')
    print()

# ==============================================================================
# Step 10: Save Results
# ==============================================================================

print("\n" + "="*60)
print("SAVING RESULTS")
print("="*60)

adata.write_h5ad(PROCESSED_DIR / "GSE116222_annotated.h5ad")
print(f"Saved: {PROCESSED_DIR}/GSE116222_annotated.h5ad")

print("""
============================================================
WEEK 4 COMPLETE
============================================================

Files generated:
  - 12_marker_umap.png: Key markers on UMAP
  - 13_marker_dotplot.png: All markers by cluster
  - 14_cluster_markers_ranked.png: DE genes per cluster
  - 15_marker_heatmap.png: Top markers heatmap
  - 16_celltype_scores.png: Cell type signature scores
  - cluster_markers.csv: Full marker gene table

Next steps:
1. Examine the dotplot to assign cell type labels
2. Validate assignments with literature
3. Create final annotation column in adata.obs
4. Proceed to Week 5: Differential Expression

Key questions to answer:
- Which clusters are stem cells? (LGR5+, ASCL2+)
- Which are goblet cells? (MUC2+, TFF3+)
- Which are the BEST4+ population? (BEST4+, OTOP2+)
- Which are inflammation-specific states?
""")

/home/vicky/miniconda3/envs/scrnaseq/lib/python3.10/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


LOADING CLUSTERED DATA


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'data/processed/GSE116222_clustered.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)